In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

def snr_cal(wv, sr)-> np.float32:
    rms_full = librosa.feature.rms(y=wv)
    noise_rms = np.mean(rms_full[:, :int(0.5*sr/512)])
    signal_rms = np.mean(rms_full)
    snr_db = 20 * np.log10(signal_rms / noise_rms)
    return snr_db

def spectral_rollof_cal(wv, sr) -> np.float64:
    rolloff = librosa.feature.spectral_rolloff(y=wv, sr=sr, roll_percent=0.85)
    return np.median(rolloff)

def spectral_flatness_cal(wv):
    flatness = librosa.feature.spectral_flatness(y=wv)
    return np.mean(flatness)

def spectral_centroid_cal(wv, sr):
    centroid = librosa.feature.spectral_centroid(y=wv, sr=sr),
    return np.median(centroid)

def zcr_cal(wv):
    zcr = librosa.feature.zero_crossing_rate(wv)
    zcr_mean = np.mean(zcr)
    zcr_var = np.var(zcr)
    return zcr_mean, zcr_var

def mfccs_cal(wv, sr, n_mfcc):
    mfccs = librosa.feature.mfcc(y=wv, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfccs, axis=1)

In [ ]:
vhp_filepath = "/Users/ac/main/amia2025-stt-benchmarking/data/audio/transcript_audio_sample.mp3"
wv, sr = librosa.load(vhp_filepath, sr=None)
print(wv.shape)
d = librosa.stft(wv)
print("transformed to complex-valued Short-Time Fourier Transform result ")
spectrogram = np.abs(d)
print("abs ensure we extract magnitude only (discards phase information)")
print("spectrogram shape: ")
print(str(spectrogram.shape))
print("FFT frequencies: Calculate frequency labels for each bin")
print("Based on sr (sample rate) and n_fft (FFT size)")
print("Frequency resolution = sr / n_fft = how wide each bin is")
print(f"In your case: {sr / 2048:.2f} Hz per bin")
print("This is a 'ruler' for the frequency axis - has nothing to do with waveform data")
freqs_scale = librosa.fft_frequencies(sr=sr, n_fft=2048)
print("frequency bin range:")
print(str((freqs_scale[0], freqs_scale[-1])))
frame_idx = 1942
print("assign arbitrary frame index: " + str(frame_idx))
frame_data = spectrogram[:, frame_idx]
print("getting spectral roll-off of the frame")
print("Calculate running total of energy from low to high frequencies")
cumulative_sum_energy = np.cumsum(frame_data)
total_energy = cumulative_sum_energy[-1]
print("total energy (the last value in cumulative_sum_energy): " +  str(total_energy))
threshold = 0.85 * total_energy
print(f"threshold (85% of total energy): {threshold}")
first_pos_above_threshold = np.where(cumulative_sum_energy >= threshold)[0][0]
print("first bin where cumulative energy ≥ 85% (apprx. position)" + str(first_pos_above_threshold))
ro = freqs_scale[first_pos_above_threshold]
print("rolloff value: " + str(ro))
# Compare with librosa
librosa_rolloff = librosa.feature.spectral_rolloff(y=wv, sr=sr, roll_percent=0.85)
librosa_value = librosa_rolloff[0, frame_idx]
print(f"Librosa rolloff: {librosa_value:.2f} Hz")
print(f"Match: {np.isclose(ro, librosa_value)}")

In [ ]:
print("=== SPECTRAL CENTROID - Manual Calculation ===")
print(f"Analyzing frame #{frame_idx}")
print(f"Frame data: energy at {len(frame_data)} frequency bins")
print("Centroid Formula: weighted average frequency")
print("  centroid = Σ(energy × frequency) / Σ(energy)")

# Manual calculation
numerator = np.sum(frame_data * freqs_scale)  # Weighted sum
denominator = total_energy

if denominator > 0:
    manual_centroid = numerator / denominator
else:
    manual_centroid = 0

print(f"Step 1: Numerator (Σ energy × frequency)")
print(f"        = {numerator:.2f}")
print()
print(f"Step 2: Denominator (Σ energy)")
print(f"        = {denominator:.2f}")
print()
print(f"Step 3: Centroid = {numerator:.2f} / {denominator:.2f}")
print(f"        = {manual_centroid:.2f} Hz")

# Compare with librosa
librosa_centroid = librosa.feature.spectral_centroid(y=wv, sr=sr)
librosa_value = librosa_centroid[0, frame_idx]

print(f"Manual calculation: {manual_centroid:.2f} Hz")
print(f"Librosa result: {librosa_value:.2f} Hz")
print(f"Match: {np.isclose(manual_centroid, librosa_value)}")
print(f"Interpretation:")
print(f"  Weighted average frequency is {manual_centroid:.0f} Hz")
if manual_centroid < 2000:
    print(f"  → DARK sound (low frequencies dominate)")
elif manual_centroid < 3500:
    print(f"  → BALANCED sound")
else:
    print(f"  → BRIGHT sound (high frequencies dominate)")

In [ ]:
print("=== VISUALIZING CENTROID AS 'CENTER OF MASS' ===")
print()

frame_idx = 1942
frame_data = spectrogram[:, frame_idx]

# Calculate cumulative energy (like roll-off)
cumulative_energy = np.cumsum(frame_data)
total_energy = cumulative_energy[-1]

# Find where 50% of energy is (should be near centroid)
half_energy = 0.5 * total_energy
bin_at_50_percent = np.where(cumulative_energy >= half_energy)[0][0]
freq_at_50_percent = freqs_scale[bin_at_50_percent]

print(f"Total energy: {total_energy:.2f}")
print(f"50% of energy: {half_energy:.2f}")
print(f"Frequency where 50% of energy is reached: {freq_at_50_percent:.2f} Hz")
print(f"Centroid (weighted average): {manual_centroid:.2f} Hz")
print()

print("Notice: 50% cutoff and centroid are SIMILAR (not exact)")
print("Centroid is like a 'balance point' - 50% of energy on each side")
print()

# Create visualization with 4 subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# ========== Plot 1: Energy Distribution (zoomed) ==========
ax1 = axes[0, 0]
ax1.fill_between(freqs_scale, frame_data, alpha=0.3, color='blue')
ax1.plot(freqs_scale, frame_data, linewidth=1, color='blue')
ax1.axvline(manual_centroid, color='red', linestyle='--', linewidth=3, 
            label=f'Centroid: {manual_centroid:.0f} Hz')
ax1.axvline(ro, color='green', linestyle='--', linewidth=2,
            label=f'Roll-off (85%): {ro:.0f} Hz')
ax1.axvline(freq_at_50_percent, color='orange', linestyle=':', linewidth=2,
            label=f'50% energy: {freq_at_50_percent:.0f} Hz')
ax1.set_xlim([0, 2000])
ax1.set_xlabel('Frequency (Hz)', fontsize=12)
ax1.set_ylabel('Energy', fontsize=12)
ax1.set_title('Energy Distribution (0-2000 Hz)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# ========== Plot 2: Cumulative Energy ==========
ax2 = axes[0, 1]
ax2.plot(freqs_scale, cumulative_energy / total_energy * 100, linewidth=2, color='purple')
ax2.axhline(50, color='orange', linestyle=':', linewidth=2, label='50% energy')
ax2.axhline(85, color='green', linestyle='--', linewidth=2, label='85% energy')
ax2.axvline(manual_centroid, color='red', linestyle='--', linewidth=3,
            label=f'Centroid: {manual_centroid:.0f} Hz')
ax2.axvline(ro, color='green', linestyle='--', linewidth=2, alpha=0.5)
ax2.set_xlim([0, 2000])
ax2.set_ylim([0, 100])
ax2.set_xlabel('Frequency (Hz)', fontsize=12)
ax2.set_ylabel('Cumulative Energy (%)', fontsize=12)
ax2.set_title('Cumulative Energy Distribution', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.fill_between(freqs_scale, 0, cumulative_energy / total_energy * 100, 
                  alpha=0.2, color='purple')

# ========== Plot 3: "Balance" Visualization ==========
ax3 = axes[1, 0]

# Create a bar chart showing energy at different frequency ranges
freq_ranges = [
    (0, 200, 'Very Low'),
    (200, 400, 'Low'),
    (400, 600, 'Mid-Low'),
    (600, 1000, 'Mid'),
    (1000, 2000, 'Mid-High'),
    (2000, 5000, 'High'),
    (5000, 24000, 'Very High')
]

energies_by_range = []
labels = []
colors_list = []

for low, high, label in freq_ranges:
    mask = (freqs_scale >= low) & (freqs_scale < high)
    energy_sum = np.sum(frame_data[mask])
    energies_by_range.append(energy_sum)
    labels.append(f"{label}\n({low}-{high}Hz)")
    
    # Color based on whether centroid is in this range
    if low <= manual_centroid < high:
        colors_list.append('red')
    else:
        colors_list.append('skyblue')

bars = ax3.bar(labels, energies_by_range, color=colors_list, edgecolor='black', linewidth=1.5)
ax3.set_ylabel('Total Energy', fontsize=12)
ax3.set_title('Energy by Frequency Range\n(Red = where centroid is)', fontsize=14, fontweight='bold')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3, axis='y')

# Add percentage labels on bars
for bar, energy in zip(bars, energies_by_range):
    height = bar.get_height()
    if energy > 0:
        pct = energy / total_energy * 100
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{pct:.1f}%', ha='center', va='bottom', fontsize=9, fontweight='bold')

# ========== Plot 4: "See-Saw" Balance Concept ==========
ax4 = axes[1, 1]

# Show energy on left vs right of centroid
left_of_centroid = frame_data[freqs_scale < manual_centroid]
right_of_centroid = frame_data[freqs_scale >= manual_centroid]

left_energy = np.sum(left_of_centroid)
right_energy = np.sum(right_of_centroid)

# Calculate "weighted distance" from centroid
freqs_left = freqs_scale[freqs_scale < manual_centroid]
freqs_right = freqs_scale[freqs_scale >= manual_centroid]

left_moment = np.sum(left_of_centroid * (manual_centroid - freqs_left))
right_moment = np.sum(right_of_centroid * (freqs_right - manual_centroid))
# moment explanation
print("\n=== Understanding 'Moment' ===")
print()
print("Moment = Energy × Distance (like see-saw physics)")
print()
print("Example:")
print("  • 10 units of energy at 100 Hz (358 Hz from centroid)")
print("    → Moment = 10 × 358 = 3,580")
print()
print("  • 1 unit of energy at 1000 Hz (541 Hz from centroid)")
print("    → Moment = 1 × 541 = 541")
print()
print("Heavy energy close to center = Light energy far from center")
print()
print(f"LEFT moment (energy × distance): {left_moment:.2f}")
print(f"RIGHT moment (energy × distance): {right_moment:.2f}")
print()
print("If LEFT ≈ RIGHT, then centroid is the correct balance point")
print(f"Ratio: {left_moment/right_moment:.3f} (close to 1.0 = balanced ✓)")
categories = ['Energy\nLeft of Centroid', 'Energy\nRight of Centroid', 
              'Weighted Moment\nLeft', 'Weighted Moment\nRight']
values = [left_energy, right_energy, left_moment/1000, right_moment/1000]  # Divide moments for scale
colors_balance = ['lightcoral', 'lightgreen', 'coral', 'green']

bars = ax4.bar(categories, values, color=colors_balance, edgecolor='black', linewidth=1.5)
ax4.set_ylabel('Value', fontsize=12)
ax4.set_title(f'Balance Check: Centroid at {manual_centroid:.0f} Hz\n(Moments should be similar)', 
              fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.1f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Print analysis
print("=" * 70)
print("ANALYSIS: What Makes 458.71 Hz the 'Center'?")
print("=" * 70)
print()
print(f"Energy LEFT of centroid (<{manual_centroid:.0f} Hz): {left_energy:.2f} ({left_energy/total_energy*100:.1f}%)")
print(f"Energy RIGHT of centroid (≥{manual_centroid:.0f} Hz): {right_energy:.2f} ({right_energy/total_energy*100:.1f}%)")
print()
print(f"Weighted moment LEFT: {left_moment:.2f}")
print(f"Weighted moment RIGHT: {right_moment:.2f}")
print(f"Ratio: {left_moment/right_moment:.3f} (should be ~1.0 for perfect balance)")
print()
print("Interpretation:")
print("  • Centroid is NOT where 50% of energy is on each side")
print("  • Centroid is where WEIGHTED DISTANCE × ENERGY balances")
print("  • Like a see-saw: heavy weight close to center = light weight far from center")
print()
print(f"For your frame: {manual_centroid:.0f} Hz is the 'balance point'")
print(f"  Most energy is BELOW {manual_centroid:.0f} Hz (dark sound)")
print(f"  Small amount of energy ABOVE {manual_centroid:.0f} Hz pulls average up slightly")

In [ ]:
print("=== SPECTRAL FLATNESS - Manual Calculation ===")
print()

print(f"Analyzing frame #{frame_idx}")
print()

print("What is Spectral Flatness?")
print("  Measures how 'noise-like' vs 'tone-like' the spectrum is")
print("  Formula: Geometric Mean / Arithmetic Mean")
print("  Range: 0 (pure tone) to 1 (white noise)")
print()

print("=== Step-by-Step Calculation ===")
print()

# Add small epsilon to avoid log(0)
epsilon = 1e-10
frame_safe = frame_data + epsilon

print("Step 1: Calculate Geometric Mean")
print("  Geometric mean = exp(mean(log(values)))")
print()

# Geometric mean
log_values = np.log(frame_safe)
mean_of_logs = np.mean(log_values)
geometric_mean = np.exp(mean_of_logs)

print(f"  log(energies) mean: {mean_of_logs:.6f}")
print(f"  exp(mean_of_logs): {geometric_mean:.6f}")
print()

print("Step 2: Calculate Arithmetic Mean")
print("  Arithmetic mean = mean(values)")
print()

arithmetic_mean = np.mean(frame_data)
print(f"  mean(energies): {arithmetic_mean:.6f}")
print()

print("Step 3: Calculate Flatness")
print("  Flatness = Geometric Mean / Arithmetic Mean")
print()

if arithmetic_mean > 0:
    manual_flatness = geometric_mean / arithmetic_mean
else:
    manual_flatness = 0

print(f"  {geometric_mean:.6f} / {arithmetic_mean:.6f} = {manual_flatness:.6f}")
print()

# Compare with librosa
librosa_flatness = librosa.feature.spectral_flatness(y=wv)
librosa_value = librosa_flatness[0, frame_idx]

print(f"✓ Manual calculation: {manual_flatness:.6f}")
print(f"✓ Librosa result: {librosa_value:.6f}")
print(f"✓ Match: {np.isclose(manual_flatness, librosa_value)}")
print()

print("=== What Does This Value Mean? ===")
print()
print(f"Flatness: {manual_flatness:.4f}")
print()

if manual_flatness < 0.1:
    interpretation = "Very TONAL (pure tone or strong harmonics)"
    quality = "Speech with clear pitch"
elif manual_flatness < 0.3:
    interpretation = "Somewhat TONAL (typical speech)"
    quality = "Normal voice with some noise"
elif manual_flatness < 0.6:
    interpretation = "MIXED (speech + noise)"
    quality = "Noisy speech or degraded audio"
else:
    interpretation = "Very NOISY (white noise / hiss)"
    quality = "Heavy tape hiss or static"

print(f"  → {interpretation}")
print(f"  → {quality}")
print()

print("Why Geometric Mean / Arithmetic Mean?")
print()
print("  • Geometric mean is ALWAYS ≤ Arithmetic mean")
print("  • For FLAT spectrum (all frequencies equal): GM = AM → Flatness = 1.0")
print("  • For PEAKED spectrum (some frequencies dominant): GM << AM → Flatness ≈ 0")
print()
print("  Think of it as:")
print("    Flatness = How close is the distribution to being uniform/flat?")

In [ ]:
print("=== ZERO-CROSSING RATE (ZCR) - Manual Calculation ===")
print()

print("What is Zero-Crossing Rate?")
print("  How often the audio waveform crosses the zero line")
print("  High ZCR = noisy/high-frequency content")
print("  Low ZCR = smooth/low-frequency content")

print("NOTE: ZCR works on WAVEFORM (time domain), not spectrogram!")

# ZCR needs the audio segment corresponding to this frame
hop_length = 512  # Default in librosa.stft()
n_fft = 2048

start_sample = frame_idx * hop_length
end_sample = start_sample + n_fft
audio_segment = wv[start_sample:end_sample]

print(f"Analyzing frame #{frame_idx}")
print(f"  Audio segment: samples {start_sample} to {end_sample}")
print(f"  Segment length: {len(audio_segment)} samples")
print(f"  Duration: {len(audio_segment) / sr * 1000:.1f} ms")

print("=== Step-by-Step Calculation ===")

print("Step 1: Get sign of each sample (+1 or -1)")
signs = np.sign(audio_segment)
print(f"  First 10 signs: {signs[:10]}")
print()

print("Step 2: Calculate differences between consecutive signs")
sign_changes = np.diff(signs)
print(f"  Sign changes (first 10): {sign_changes[:10]}")
print("  (Non-zero values = zero-crossing happened)")
print()

print("Step 3: Count zero crossings")
# Each crossing is counted twice (from +1 to -1 OR from -1 to +1)
# So we divide by 2
zero_crossings = np.sum(np.abs(sign_changes)) / 2
print(f"  Total zero crossings: {int(zero_crossings)}")
print()

print("Step 4: Normalize by segment length")
zcr_manual = zero_crossings / len(audio_segment)
print(f"  ZCR = {int(zero_crossings)} / {len(audio_segment)}")
print(f"      = {zcr_manual:.6f}")
print()

# Compare with librosa
librosa_zcr = librosa.feature.zero_crossing_rate(wv)
librosa_value = librosa_zcr[0, frame_idx]

print(f"✓ Manual calculation: {zcr_manual:.6f}")
print(f"✓ Librosa result: {librosa_value:.6f}")
print(f"✓ Match: {np.isclose(zcr_manual, librosa_value)}")
print()

print("=== What Does This Value Mean? ===")
print()
print(f"ZCR: {zcr_manual:.4f}")
print()

if zcr_manual < 0.02:
    interpretation = "Low ZCR → Smooth, low-frequency sound"
    example = "Deep voice, bass, vowels"
elif zcr_manual < 0.05:
    interpretation = "Medium ZCR → Typical speech"
    example = "Normal conversation"
elif zcr_manual < 0.10:
    interpretation = "High ZCR → Noisy or high-frequency sound"
    example = "Consonants (s, f, sh), noise"
else:
    interpretation = "Very high ZCR → Very noisy"
    example = "Static, white noise, hiss"

print(f"  → {interpretation}")
print(f"  → Example: {example}")

print("Why does ZCR indicate frequency content?")
print()
print("  Low frequency (100 Hz):")
print("    Completes 100 cycles/second")
print("    Crosses zero 200 times/second")
print("    ZCR = 200 / 48000 = 0.0042")
print()
print("  High frequency (5000 Hz):")
print("    Completes 5000 cycles/second") 
print("    Crosses zero 10,000 times/second")
print("    ZCR = 10000 / 48000 = 0.208")
print()
print("  More zero crossings = Higher frequency content!")

In [ ]:
print("=== ZCR MEAN AND VARIANCE - Across All Frames ===")
print()

# Calculate ZCR for all frames
all_zcr = librosa.feature.zero_crossing_rate(wv)
print(f"ZCR calculated for all frames: {all_zcr.shape}")
print(f"  (1 row × {all_zcr.shape[1]} time frames)")
print()

print("What are ZCR Mean and Variance?")
print()
print("  ZCR Mean = Average zero-crossing rate across entire audio")
print("    → Tells you overall frequency content")
print()
print("  ZCR Variance = How much ZCR varies across time")
print("    → Tells you if audio is consistent or changing")
print()

# Calculate mean and variance
zcr_mean = np.mean(all_zcr)
zcr_variance = np.var(all_zcr)

print(f"ZCR Mean: {zcr_mean:.6f}")
print(f"ZCR Variance: {zcr_variance:.8f}")
print()

print("=== Interpretation ===")
print()

print(f"Mean ({zcr_mean:.4f}):")
if zcr_mean < 0.02:
    print("  → Overall LOW frequency content")
    print("  → Audio is predominantly low-pitched (bass-heavy)")
elif zcr_mean < 0.05:
    print("  → Overall BALANCED frequency content")
    print("  → Mix of low and high frequencies")
else:
    print("  → Overall HIGH frequency content")
    print("  → Audio is noisy or high-pitched")
print()

print(f"Variance ({zcr_variance:.6f}):")
if zcr_variance < 0.0001:
    print("  → Very STABLE (consistent ZCR throughout)")
    print("  → Audio doesn't change much (monotone or continuous noise)")
elif zcr_variance < 0.001:
    print("  → Somewhat STABLE")
    print("  → Normal speech variation")
else:
    print("  → Very VARIABLE (ZCR changes a lot)")
    print("  → Dynamic audio with varying content")
print()

print("Why does variance matter?")
print()
print("  Low variance:")
print("    • Steady hum/tone")
print("    • Continuous speech without pauses")
print("    • Background noise")
print()
print("  High variance:")
print("    • Speech with pauses/silence")
print("    • Music with dynamics")
print("    • Switching between vowels (low ZCR) and consonants (high ZCR)")

In [ ]:
# Plot ZCR variation over time
plt.figure(figsize=(14, 6))

plt.subplot(2, 1, 1)
time_frames = np.arange(all_zcr.shape[1]) * hop_length / sr
plt.plot(time_frames, all_zcr[0], linewidth=0.5, alpha=0.7)
plt.axhline(zcr_mean, color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {zcr_mean:.4f}')
plt.axhline(zcr_mean + np.sqrt(zcr_variance), color='orange', 
            linestyle=':', linewidth=1.5, label='Mean + 1 Std Dev')
plt.axhline(zcr_mean - np.sqrt(zcr_variance), color='orange',
            linestyle=':', linewidth=1.5, label='Mean - 1 Std Dev')
plt.xlabel('Time (seconds)', fontsize=12)
plt.ylabel('Zero-Crossing Rate', fontsize=12)
plt.title('ZCR Over Time - Shows Frequency Content Variation', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim([100, 110])  # Zoom to 10 seconds for detail

plt.subplot(2, 1, 2)
plt.hist(all_zcr[0], bins=100, edgecolor='black', alpha=0.7, color='skyblue')
plt.axvline(zcr_mean, color='red', linestyle='--', linewidth=2,
            label=f'Mean: {zcr_mean:.4f}')
plt.xlabel('Zero-Crossing Rate', fontsize=12)
plt.ylabel('Frequency (number of frames)', fontsize=12)
plt.title('ZCR Distribution - Most Common Values', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("What you're seeing:")
print()
print("  Top plot: ZCR changes over time")
print("    • Spikes = high-frequency moments (consonants, noise)")
print("    • Dips = low-frequency moments (vowels, pauses)")
print()
print("  Bottom plot: Distribution of ZCR values")
print("    • Peak shows most common ZCR")
print("    • Spread shows variability")

In [ ]:
print("=" * 70)
print(f"COMPLETE ANALYSIS: Frame {frame_idx}")
print("=" * 70)
print()

# Calculate all metrics for this frame
frame_data = spectrogram[:, frame_idx]

# Roll-off
cumsum = np.cumsum(frame_data)
threshold = 0.85 * cumsum[-1]
rolloff_bin = np.where(cumsum >= threshold)[0][0]
rolloff = freqs_scale[rolloff_bin]

# Centroid
centroid = np.sum(frame_data * freqs_scale) / np.sum(frame_data)

# Flatness
frame_safe = frame_data + 1e-10
geometric_mean = np.exp(np.mean(np.log(frame_safe)))
arithmetic_mean = np.mean(frame_data)
flatness = geometric_mean / arithmetic_mean

# ZCR for this frame
start_sample = frame_idx * hop_length
audio_segment = wv[start_sample:start_sample + n_fft]
signs = np.sign(audio_segment)
zcr_frame = np.sum(np.abs(np.diff(signs))) / 2 / len(audio_segment)

print(f"{'Metric':<25} {'Value':<20} {'Interpretation'}")
print("-" * 70)
print(f"{'Spectral Roll-off':<25} {rolloff:>10.2f} Hz       Bandwidth cutoff")
print(f"{'Spectral Centroid':<25} {centroid:>10.2f} Hz       Brightness/darkness")
print(f"{'Spectral Flatness':<25} {flatness:>10.6f}          Tonality (0=tone, 1=noise)")
print(f"{'Zero-Crossing Rate':<25} {zcr_frame:>10.6f}          Frequency content")
print()
print("-" * 70)
print("Overall Assessment:")
print("-" * 70)

if rolloff < 1000:
    print(f"  Roll-off ({rolloff:.0f} Hz): SEVERELY LIMITED bandwidth")
elif rolloff < 5000:
    print(f"  Roll-off ({rolloff:.0f} Hz): LIMITED bandwidth")
else:
    print(f"  Roll-off ({rolloff:.0f} Hz): Good bandwidth")

if centroid < 1500:
    print(f"  Centroid ({centroid:.0f} Hz): VERY DARK sound")
elif centroid < 2500:
    print(f"  Centroid ({centroid:.0f} Hz): Dark sound")
else:
    print(f"  Centroid ({centroid:.0f} Hz): Balanced/bright sound")

if flatness < 0.1:
    print(f"  Flatness ({flatness:.4f}): Very tonal (clear pitch)")
elif flatness < 0.3:
    print(f"  Flatness ({flatness:.4f}): Somewhat tonal (normal speech)")
else:
    print(f"  Flatness ({flatness:.4f}): Noisy")

if zcr_frame < 0.02:
    print(f"  ZCR ({zcr_frame:.4f}): Low-frequency content")
elif zcr_frame < 0.05:
    print(f"  ZCR ({zcr_frame:.4f}): Balanced frequency content")
else:
    print(f"  ZCR ({zcr_frame:.4f}): High-frequency/noisy content")

print()
print("=" * 70)
print("This frame is characteristic of 'BROKEN RECORD' degraded audio!")
print("=" * 70)

In [ ]:
print("=== VISUALIZING AUDIO QUALITY ===")
print()

# Load audio
y, sr = librosa.load(vhp_filepath, sr=None)

# Create figure with 3 plots
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# ========== Plot 1: Waveform ==========
ax1 = axes[0]
times = np.arange(len(y)) / sr
ax1.plot(times, y, linewidth=0.3, alpha=0.7, color='blue')
ax1.set_xlim([0, len(y)/sr])
ax1.set_xlabel('Time (seconds)', fontsize=12)
ax1.set_ylabel('Amplitude', fontsize=12)
ax1.set_title('Audio Waveform - Time Domain', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Show a 10-second excerpt
# ax1.set_xlim([100, 110])  # Zoom to 100-110 seconds

# ========== Plot 2: Spectrogram ==========
ax2 = axes[1]
D = librosa.stft(y)
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

img = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='hz', 
                               ax=ax2, cmap='viridis')
ax2.set_ylim([0, 3000])  # Only show 0-3000 Hz to see detail
# ax2.set_xlim([100, 110])  # Same time window as waveform
ax2.set_title('Spectrogram (0-3000 Hz) - DARK AUDIO = Energy only at bottom', 
              fontsize=14, fontweight='bold')
fig.colorbar(img, ax=ax2, format='%+2.0f dB')

# ========== Plot 3: Full Frequency Range Spectrogram ==========
ax3 = axes[2]
img2 = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='hz',
                                ax=ax3, cmap='viridis')
# ax3.set_xlim([100, 110])  # Same time window
ax3.set_title('Spectrogram (Full Range 0-24kHz) - Notice empty space at top = LIMITED BANDWIDTH',
              fontsize=14, fontweight='bold')
fig.colorbar(img2, ax=ax3, format='%+2.0f dB')

plt.tight_layout()
plt.show()

print("What you're seeing:")
print("  ✓ Spectrogram shows frequency (vertical) over time (horizontal)")
print("  ✓ Brightness = energy at that frequency")
print("  ✓ DARK AUDIO = bright colors only at BOTTOM (low frequencies)")
print("  ✓ BAD QUALITY = empty space at top (high frequencies missing)")

In [ ]:
# Calculate roll-off for every 100th frame (to see variation over time)
frame_indices = range(0, spectrogram.shape[1], 100)
rolloffs_over_time = []

for frame_idx in frame_indices:
    frame_data = spectrogram[:, frame_idx]
    cumsum = np.cumsum(frame_data)
    threshold = 0.85 * cumsum[-1]
    
    bins_above = np.where(cumsum >= threshold)[0]
    if len(bins_above) > 0:
        bin_idx = bins_above[0]
        rolloff_freq = freqs_scale[bin_idx]
        rolloffs_over_time.append(rolloff_freq)

# Plot
plt.figure(figsize=(14, 5))
plt.plot(rolloffs_over_time, linewidth=0.8, alpha=0.7)
plt.axhline(562.5, color='red', linestyle='--', linewidth=2, 
            label='Librosa median: 562.5 Hz')
plt.axhline(np.median(rolloffs_over_time), color='green', linestyle='--', linewidth=2,
            label=f'Our median: {np.median(rolloffs_over_time):.0f} Hz')
plt.xlabel('Frame Index (every 100th frame)')
plt.ylabel('Roll-off Frequency (Hz)')
plt.title('Roll-off Variation Across Time')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim([0, 5000])  # Limit y-axis to see detail
plt.show()

print(f"Our calculated median from sampled frames: {np.median(rolloffs_over_time):.2f} Hz")
print(f"Librosa's median from ALL frames: 562.5 Hz")
print(f"Difference: {abs(np.median(rolloffs_over_time) - 562.5):.2f} Hz")

In [ ]:
plt.plot(cumulative_sum_energy)
plt.show()

In [ ]:
vhp_filepath = "/Users/ac/main/amia2025-stt-benchmarking/data/audio/transcript_audio_sample.mp3"
wv, sr = librosa.load(vhp_filepath, sr=None)
# librosa.display.waveshow(wv, sr=sr)
print("SNR: " + str(round(snr_cal(wv, sr), 4)))
print("Spectral Roll-off (Hz): " + str(round(spectral_rollof_cal(wv, sr), 4)))
print("Spectral Flatness: " + str(round(spectral_flatness_cal(wv), 4)))
print("Spectral Centroid: " + str(round(spectral_centroid_cal(wv, sr), 4)))
zcr_mean, zcr_var = zcr_cal(wv)
print("ZCR (mean): " + str(round(zcr_mean, 4)))
print("ZCR (var): " + str(round(zcr_var, 4)))
print("MFCCS: ")
plt.plot(mfccs_cal(wv, sr, 13))


In [ ]:
nbc_filepath = "/Users/ac/main/amia2025-stt-benchmarking/data/audio/208-192.mp3"
wv, sr = librosa.load(nbc_filepath, sr=None)
# librosa.display.waveshow(wv, sr=sr)
print("SNR: " + str(round(snr_cal(wv, sr), 4)))
print("Spectral Roll-off (Hz): " + str(round(spectral_rollof_cal(wv, sr), 4)))
print("Spectral Flatness: " + str(round(spectral_flatness_cal(wv), 4)))
print("Spectral Centroid: " + str(round(spectral_centroid_cal(wv, sr), 4)))
zcr_mean, zcr_var = zcr_cal(wv)
print("ZCR (mean): " + str(round(zcr_mean, 4)))
print("ZCR (var): " + str(round(zcr_var, 4)))
print("MFCCS: ")
plt.plot(mfccs_cal(wv, sr, 13))